# Getting data from aquamonitor

In [1]:
import requests
import getpass
import json
import xmltodict
from fabric import Connection
import datetime
import pandas as pd

In [2]:
username = getpass.getpass('username: ')
password = getpass.getpass('password: ')

login_params = {'username' : username, 'password' : password}

username:  ···
password:  ··············


In [3]:
host = "http://www.aquamonitor.no/"

In [4]:
#Logging in
login_url =  host + "AquaServices" + "/WebServices/LoginService.asmx/AuthenticateUser"

r = requests.get(login_url,params=login_params)
display(r.status_code)
login_dict = xmltodict.parse(r.text)
token = login_dict['UserCargo']['Token']
key = login_dict['UserCargo']['Key']

200

In [5]:
project_codes = {'Langtjern' : 4300, 'Birkenes' : 2898} #The project should include the stations we are interested in

In [6]:
# Getting all data for available projects at Langtjern and Birkenes
#Filtering stations in project by name
station_codes = {}
for i,j in project_codes.items():
    projectsurl = 'AquaServices/api/projects/{projectId}/Stations'.format(projectId=j) #9906
    r = requests.get(host + projectsurl,cookies = dict(aqua_key=token))
#     display(r.json())
    station_codes.update({k['Name'] : k['Id'] for k in r.json() if i in k['Name']})

station_codes

{'Bøye i Langtjern': 50472,
 'Innløpet LAE03 - Langtjern': 63099,
 'Utløp av Langtjern': 51356,
 'Utløp av Langtjern - jord': 51357,
 'Værstasjon ved Langtjern': 62040,
 'Utløp av Langtjern - luft': 51358,
 'Birkenes': 37927}

In [7]:
#Creating directory to store aquamonitor data
from fabric import Connection
with Connection('localhost') as c:
    c.local('''rm -rf aquamonitor && mkdir aquamonitor''')

In [8]:
#Creating data file
from datetime import date
archiveurl = host + 'AquaServices/files/archive/{aid}'
datafileurl = host + 'AquaServices/files/datafile'

today = date.today().strftime('%d.%m.%Y')
startDate = '01.01.1900' #So as to get the entire timeseries


datawhere = '''sample_date>={startDate} and sample_date<={endDate}'''.format(startDate=startDate,
                                                                             endDate=today)

params = {"Expires": (date.today() + datetime.timedelta(days=1)).strftime('%Y.%m.%d'),
       "Title": '',
       "Files":[{"Filename": '',
                "ContentType":"text/csv"}],
       "Definition":{"Format":"csv",
                     "StationIds": [ '' ],
                     "DataWhere": datawhere
                    }
        }

for i,j in station_codes.items():
    params['Title'] = i.replace(' ','_')
    filename = params['Title'] + '.csv'
    params['Files'][0]['Filename'] = filename
    params['Definition']['StationIds'] = [j]
    display('Processing station {}'.format(i))
    #Create datafile in server
    r = requests.post(datafileurl,json=params,cookies=dict(aqua_key=token))
    currentId = r.json()['Id']
#     display(currentId)
    #Generating archive
    archived = False
    while not archived:
#         display(archiveurl.format(aid=currentId))
        resp = requests.get(archiveurl.format(aid=currentId),cookies = dict(aqua_key=token))
        archived = resp.json().get('Archived')
    #Downloading the archive
    fr = requests.get(archiveurl.format(aid=currentId) + '/' +filename ,cookies = dict(aqua_key=token))
    with open('./aquamonitor/{}'.format(filename),'wb') as fd:
        for chunk in fr.iter_content(chunk_size=256):
            fd.write(chunk)
        



'Processing station Bøye i Langtjern'

'Processing station Innløpet LAE03 - Langtjern'

'Processing station Utløp av Langtjern'

'Processing station Utløp av Langtjern - jord'

'Processing station Værstasjon ved Langtjern'

'Processing station Utløp av Langtjern - luft'

'Processing station Birkenes'

In [9]:
df = pd.read_csv('./aquamonitor/Birkenes.csv',delimiter=';',encoding='utf-16')
df

,ProjectId,ProjectName,StationId,StationCode,StationName,SampleDate_dato,SampleDate_tid,Depth1,Depth2,Al_µg/l,...,NO3-N_µg/l,pH,PO4-P_µg/l,SO4_mg/L,SO4_Old_mg/l,TOC_mg/l,TOTN_µg/l,TOTN (est.)_µg/l,TOTN (old EF)_µg/l,TOTP_µg/l
0,2898,FELTFO,37927,BIE01,Birkenes,20.07.1972,9:00:00,0.0,0.0,NaN,...,10.0,5.66,NaN,NaN,4.3,NaN,NaN,NaN,NaN,NaN
1,2898,FELTFO,37927,BIE01,Birkenes,21.07.1972,8:00:00,0.0,0.0,NaN,...,30.0,5.44,NaN,NaN,4.4,NaN,NaN,NaN,NaN,NaN
2,2898,FELTFO,37927,BIE01,Birkenes,22.07.1972,8:00:00,0.0,0.0,NaN,...,30.0,5.40,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN
3,2898,FELTFO,37927,BIE01,Birkenes,23.07.1972,8:00:00,0.0,0.0,NaN,...,30.0,5.47,NaN,NaN,4.7,NaN,NaN,NaN,NaN,NaN
4,2898,FELTFO,37927,BIE01,Birkenes,24.07.1972,8:00:00,0.0,0.0,NaN,...,50.0,5.33,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15615,12470,Sur nedbør - Feltfo 2019,37927,BIE01,Birkenes,01.02.2021,0:00:00,0.0,0.0,NaN,...,120.0,5.09,< 1,1.68,NaN,3.2,270.0,NaN,NaN,1.0
15616,12470,Sur nedbør - Feltfo 2019,37927,BIE01,Birkenes,08.02.2021,0:00:00,0.0,0.0,NaN,...,150.0,5.19,< 1,2.03,NaN,2.9,250.0,NaN,NaN,2.0
15617,12470,Sur nedbør - Feltfo 2019,37927,BIE01,Birkenes,15.02.2021,0:00:00,0.0,0.0,NaN,...,140.0,5.66,< 1,1.84,NaN,2.9,310.0,NaN,NaN,2.0
15618,12470,Sur nedbør - Feltfo 2019,37927,BIE01,Birkenes,01.03.2021,0:00:00,0.0,0.0,NaN,...,150.0,4.83,< 1,1.52,NaN,3.6,290.0,NaN,NaN,2.0
